## Using the results I got from my Kmeans clustering, I ran various models (naive bayes, logistic regression, and neural network) to see if I can classify the news titles into the clusters from the Kmeans algorithm. 

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%cd gdrive/MyDrive/

/content/gdrive/MyDrive


# Import necessary libararies

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd

# Load in DF

In [15]:
df = pd.read_csv('Eluvio_DS_Challenge.csv')
df['title'] = pd.read_csv('kmeans_df.csv')['title']
df['kmeans12'] = pd.read_csv('kmeans_df.csv')['kmeans12']
df.shape
df.head()

,time_created,date_created,up_votes,down_votes,title,over_18,author,category,kmeans12
0,1201232046,2008-01-25,3,0,Scores killed Pakistan clashes,False,polar,worldnews,9
1,1201232075,2008-01-25,2,0,Japan resumes refuelling mission,False,polar,worldnews,3
2,1201232523,2008-01-25,3,0,presses Egypt Gaza border,False,polar,worldnews,3
3,1201233290,2008-01-25,1,0,Jump-start economy Give health care,False,fadi420,worldnews,3
4,1201274720,2008-01-25,4,0,Council Europe bashes UN terror blacklist,False,mhermans,worldnews,3


In [29]:
df['title'] = df['title'].values.astype('U')

# Label Encoding

In [30]:
# Create a label encoder object
le = preprocessing.LabelEncoder()

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509236 entries, 0 to 509235
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   time_created  509236 non-null  int64 
 1   date_created  509236 non-null  object
 2   up_votes      509236 non-null  int64 
 3   down_votes    509236 non-null  int64 
 4   title         509236 non-null  object
 5   over_18       509236 non-null  bool  
 6   author        509236 non-null  object
 7   category      509236 non-null  object
 8   kmeans12      509236 non-null  int64 
dtypes: bool(1), int64(4), object(4)
memory usage: 31.6+ MB


In [32]:
# Fit the encoder to the pandas column
le.fit(df.kmeans12)

LabelEncoder()

In [33]:
targets = list(le.classes_)
print(targets)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [34]:
# Apply the fitted encoder to the pandas column
le.transform(df['kmeans12'])

array([ 9,  3,  3, ...,  3,  3, 10])

# Train Test Split

In [35]:
X = df.title
y = df.kmeans12

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [36]:
y_test

369904     3
32341      1
292830    10
415889     3
282176     8
          ..
241591     3
43502     11
403375     3
341786     3
389355     3
Name: kmeans12, Length: 152771, dtype: int64

# Naive Bayes Classifier

In [37]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [38]:
%%time

y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.69144012934392
CPU times: user 2 s, sys: 7.44 ms, total: 2.01 s
Wall time: 2.02 s


# Logistic Regression Classifier

In [39]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [40]:
%%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.9921385603288583
CPU times: user 1.93 s, sys: 2.66 ms, total: 1.94 s
Wall time: 1.95 s


# Bag of Words Keras Neural Network

In [41]:
import itertools
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

In [43]:
#train test split
X = df.title
y = df.kmeans12

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 40)

In [44]:
#tokenize the unique words in the vocab
max_words = 1000

#limit the vocab to the top words (max_words)
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

#creates a word index lookup of the vocab
tokenize.fit_on_texts(X_train) # only fit on train

In [45]:
#create matrix to pass into the neural network
x_train = tokenize.texts_to_matrix(X_train)
x_test = tokenize.texts_to_matrix(X_test)

In [46]:
#encode labels and fit to training
encoder = LabelEncoder()
encoder.fit(y_train)

#transform labels
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [47]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [48]:
batch_size = 32
epochs = 5

In [49]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [50]:
#set optimizer and loss, test for accuracy
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [51]:
#run the model
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
10026/10026 [==============================] - 44s 4ms/step - loss: 0.2029 - accuracy: 0.9434 - val_loss: 0.0637 - val_accuracy: 0.9784
Epoch 2/10
10026/10026 [==============================] - 43s 4ms/step - loss: 0.0570 - accuracy: 0.9808 - val_loss: 0.0647 - val_accuracy: 0.9792
Epoch 3/10
10026/10026 [==============================] - 43s 4ms/step - loss: 0.0426 - accuracy: 0.9858 - val_loss: 0.0677 - val_accuracy: 0.9782
Epoch 4/10
10026/10026 [==============================] - 42s 4ms/step - loss: 0.0357 - accuracy: 0.9881 - val_loss: 0.0709 - val_accuracy: 0.9781
Epoch 5/10
10026/10026 [==============================] - 42s 4ms/step - loss: 0.0297 - accuracy: 0.9903 - val_loss: 0.0742 - val_accuracy: 0.9779
Epoch 6/10
10026/10026 [==============================] - 43s 4ms/step - loss: 0.0256 - accuracy: 0.9914 - val_loss: 0.0778 - val_accuracy: 0.9783
Epoch 7/10
10026/10026 [==============================] - 45s 5ms/step - loss: 0.0227 - accuracy: 0.9927 - val_loss: 0

In [52]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

4775/4775 [==============================] - 10s 2ms/step - loss: 0.1024 - accuracy: 0.9768
Test accuracy: 0.9768477082252502


In [53]:
score

[0.10236397385597229, 0.9768477082252502]